<a href="https://colab.research.google.com/github/cristinapopescu19/Projects/blob/AI/Proiect_lab_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

def succ(s):
    succesorii = [] # lista cu succesorii
    for i in range(9):
        for j in range(9):
            if s[i][j] == 0: # daca se gaseste o celula cu valoarea 0
                for k in range(1, 10): # se incearca valorile de la 1 la 9 pentru a completa celula
                    new_s = np.copy(s) # se creeaza o noua matrice cu aceleasi valori ca s
                    new_s[i][j] = k # se completeaza celula cu valoarea k
                    if valid_sudoku(new_s): # se verifica daca noua matrice este valida si daca da, se adauga in lista de succesori
                        succesorii.append(new_s)
    return succesorii # daca nu s-au gasit celule cu valoarea 0, se returneaza lista goala, nu mai putem genera niciun succesor

    return True

def valid(s):
    for i in range(9):
        for j in range(9):
            if s[i][j] == 0:
                continue
            for k in range(9):
                if k != j and s[i][k] == s[i][j]:
                    return False
                if k != i and s[k][j] == s[i][j]:
                    return False
            for m in range(i//3*3, i//3*3+3):
                for n in range(j//3*3, j//3*3+3):
                    if m != i and n != j and s[m][n] == s[i][j]:
                        return False
    return True


def rezolva_s(s, algoritm):
    if not valid(s):
        return None

    if algoritm == "DFS":
       graph_dfs = {} # adauga aceasta linie
    def dfs(graph, start, visited=None):
        if visited is None:
            visited = set()
        stack=[start]
        visited.add(start)

        while stack:
            nod_curent = stack.pop()
            if final(nod_curent):
                return nod_curent
            succesorii = succ(nod_curent)
            for succesor in succesorii:
                if succesor not in visited:
                    visited.add(succesor)
                    stack.append(succesor)
                    graph_dfs[tuple(map(tuple, nod_curent))].append(tuple(map(tuple, succesor))) # inlocuieste graph cu graph_dfs


def final(s):
    if not valid(s):
        return False
    for i in range(9): # parcurgem toate randurile
        for j in range(9): # parcurgem toate coloanele
            if s[i][j] == 0: # daca gasim o celula cu valoarea 0, intoarcem False
                return False
    return True # daca nu am gasit nicio celula cu valoarea 0, intoarcem True


s = np.array([[5,0,7,2,0,0,0,9,0],
                   [0,0,6,0,3,0,7,0,1],
                   [4,0,0,0,0,0,0,6,0],
                   [1,0,0,4,9,0,0,0,7],
                   [0,0,0,5,0,8,0,0,0],
                   [8,0,0,0,2,7,0,0,5],
                   [0,7,0,0,0,0,0,0,9],
                   [2,0,9,0,8,0,6,0,0],
                   [0,4,0,0,0,9,3,0,8]])

rezultat = rezolva_s(s, "DFS")
print(rezultat)







None


8-puzzle

In [ ]:
from queue import PriorityQueue

# h1 - numarul de piese care nu sunt la locul lor
def h1(state):
    count = 0
    for i in range(len(state)): #for parcurge numerele de la 0 până la len(stare) - 1
        if state[i] != i:
            count += 1
    return count #Funcția h1 returnează valoarea stocată în variabila count

# h2 - suma distantelor Manhattan pentru fiecare piesa
def h2(state):
    dist = 0
    for i in range(len(state)):
        if state[i] != i:
            x1, y1 = divmod(i, 3) #Se calculează coordonatele x și y ale poziției actuale a elementului
            x2, y2 = divmod(state[i], 3) #Se calculează coordonatele x și y ale poziției la care ar trebui să fie elementul
            dist += abs(x1 - x2) + abs(y1 - y2) #se calculeaza suma diferențelor absolute între coordonatele x și y ale pozitiei curente și celei țintă
    return dist

def expand(state):
# generează toate stările 'child' valide pentru starea dată. Se găsește indicele poziției elementului 0
#(reprezentând spațiul gol),
#iar pentru fiecare index valid al unei mișcări, se face o copie a stării și se realizează respectiva mutare.
    children = []
    zero = state.index(0) #indexul pozitiei spatiului gol e determinat cu functia index()
    if zero not in [0, 1, 2]: # mutare sus, iar dacă elementul 0 nu se află pe prima linie, se executa următoarea instrucțiune.
        child = list(state) #dacă spațiul gol poate fi mutat în sus, se creează o copie a stării curente a jocului, reprezentată de lista 'child'.
        child[zero], child[zero - 3] = child[zero - 3], child[zero] #elementele de pe poziția libera și de pe poziția de trei pătrățele mai sus sunt interschimbate
        children.append(tuple(child)) #starea nouă a jocului este adăugată în lista 'children' ca tuplu

    if zero not in [0, 3, 6]: # mutare stanga, verifică dacă spațiul gol poate fi mutat spre stânga și, dacă este posibil, face această mutare.
        child = list(state)
        child[zero], child[zero - 1] = child[zero - 1], child[zero] #elementele de pe poziția spațiului gol și de pe poziția din stânga sa sunt schimbate între ele
        children.append(tuple(child)) #starea nouă a jocului este adăugată în lista 'children' ca tuplu

    if zero not in [6, 7, 8]: # mutare jos, dacă spațiul gol se află deja în ultimele trei rânduri ale tablei, atunci nu poate fi mutat în jos.
        child = list(state)
        child[zero], child[zero + 3] = child[zero + 3], child[zero] #elementele de pe poziția spațiului gol și de pe poziția de sub acesta sunt interschimbate
        children.append(tuple(child)) #starea nouă a jocului este adăugată în lista 'children' ca tuplu

    if zero not in [2, 5, 8]: # mutare dreapta, dacă spațiul gol se află deja în ultimele trei coloane ale tablei, atunci nu poate fi mutat în dreapta.
        child = list(state)
        child[zero], child[zero + 1] = child[zero + 1], child[zero] #elementele de pe poziția spațiului gol și de pe poziția din dreapta acestuia sunt interschimbate
        children.append(tuple(child)) #starea nouă a jocului este adăugată în lista 'children' ca tuplu
    return children

def solve_puzzle(start, goal, heuristic):
    if start == goal: #verifică dacă starea de pornire este deja egală cu starea scop. În acest caz, returneaza o listă goală.
        return []
    if heuristic == 'h1': #verifică ce euristică a fost specificată
        f = h1
    elif heuristic == 'h2':
        f = h2
    frontier = PriorityQueue() #Frontiera este o coadă de priorități care va fi utilizată pentru a menține toate stările care încă nu au fost explorate.
    frontier.put((0, [start], 0)) # initializare cost = 0
    explored = set() #set care conține toate stările deja explorate
    while frontier: #cat timp există elemente în frontieră
        _, path, cost = frontier.get() # extrage nodul cu cel mai mic cost, descompune tuplul returnat de 'get' al frontierei
        state = path[-1] #setează variabila "state" să fie ultima stare din lista "path"
        if state == goal: #daca starea curenta este starea scop, returneaza lista de stări (path) prin care am trecut
            return path
        explored.add(state)
        for child in expand(state): # pentru fiecare 'child' nou generat, verifică dacă acesta nu se află deja în setul de explorate.
            if child not in explored: #daca nu,
                new_path = path + [child] # creeaza o nouă cale care include acest copil și o adauga la coada de priorități.
                new_cost = cost + 1
                frontier.put((new_cost, new_path, new_cost)) # adaugă nodul în coada + costul său total
    return None

def bfs(start, goal):
    if start == goal:
        return []
    frontier = [[start]] #inițializarea listei 'frontier' cu starea inițială sub formă de listă de liste, unde fiecare listă reprezintă o cale posibilă
    explored = set() #inițializarea 'explored' cu starea inițială
    while frontier:
        path = frontier.pop(0) #extrage prima cale posibilă din lista 'frontier' și o atribuie variabilei 'path'
        state = path[-1] #extrage ultima stare din calea curentă și o atribuim variabilei 'state'
        if state == goal: #dacă starea curentă este identică cu starea scop,
            return path #returneaza calea curentă
        explored.add(state) #adauga starea curentă în 'explored'
        for child in expand(state): #trece prin toate stările copil posibile pentru starea curentă
            if child not in explored: #verifică dacă starea copil nu a fost explorată încă
                new_path = path + [child] #adauga starea copil în calea curentă pentru a forma o nouă cale
                frontier.append(new_path) #adăugăm calea posibilă nou formată în 'frontier'
    return None

start = (1, 3, 2, 4, 5, 6, 8, 7, 0)
goal = (1, 2, 3, 4, 5, 6, 7, 8, 0)

v1 = solve_puzzle(start, goal, 'h1')
v2 = solve_puzzle(start, goal, 'h2')
v3 = bfs(start, goal)

print(v1)
print(v2)
print(v3)




[(1, 3, 2, 4, 5, 6, 8, 7, 0), (1, 3, 2, 4, 5, 0, 8, 7, 6), (1, 3, 0, 4, 5, 2, 8, 7, 6), (1, 0, 3, 4, 5, 2, 8, 7, 6), (0, 1, 3, 4, 5, 2, 8, 7, 6), (4, 1, 3, 0, 5, 2, 8, 7, 6), (4, 1, 3, 5, 0, 2, 8, 7, 6), (4, 1, 3, 5, 7, 2, 8, 0, 6), (4, 1, 3, 5, 7, 2, 0, 8, 6), (4, 1, 3, 0, 7, 2, 5, 8, 6), (4, 1, 3, 7, 0, 2, 5, 8, 6), (4, 1, 3, 7, 2, 0, 5, 8, 6), (4, 1, 3, 7, 2, 6, 5, 8, 0), (4, 1, 3, 7, 2, 6, 5, 0, 8), (4, 1, 3, 7, 2, 6, 0, 5, 8), (4, 1, 3, 0, 2, 6, 7, 5, 8), (0, 1, 3, 4, 2, 6, 7, 5, 8), (1, 0, 3, 4, 2, 6, 7, 5, 8), (1, 2, 3, 4, 0, 6, 7, 5, 8), (1, 2, 3, 4, 5, 6, 7, 0, 8), (1, 2, 3, 4, 5, 6, 7, 8, 0)]
[(1, 3, 2, 4, 5, 6, 8, 7, 0), (1, 3, 2, 4, 5, 0, 8, 7, 6), (1, 3, 0, 4, 5, 2, 8, 7, 6), (1, 0, 3, 4, 5, 2, 8, 7, 6), (0, 1, 3, 4, 5, 2, 8, 7, 6), (4, 1, 3, 0, 5, 2, 8, 7, 6), (4, 1, 3, 5, 0, 2, 8, 7, 6), (4, 1, 3, 5, 7, 2, 8, 0, 6), (4, 1, 3, 5, 7, 2, 0, 8, 6), (4, 1, 3, 0, 7, 2, 5, 8, 6), (4, 1, 3, 7, 0, 2, 5, 8, 6), (4, 1, 3, 7, 2, 0, 5, 8, 6), (4, 1, 3, 7, 2, 6, 5, 8, 0), (4, 1, 3, 7,

In [ ]:
s1 = (1, 3, 2, 4, 5, 6, 8, 7, 0)
s2 = (1, 3, 2, 4, 5, 0, 8, 7, 6)
s3 = (1, 3, 0, 4, 5, 2, 8, 7, 6)
s4 = (1, 0, 3, 4, 5, 2, 8, 7, 6)
s5 = (0, 1, 3, 4, 5, 2, 8, 7, 6)
s6 = (4, 1, 3, 0, 5, 2, 8, 7, 6)
s7 = (4, 1, 3, 5, 0, 2, 8, 7, 6)
s8 = (4, 1, 3, 5, 7, 2, 8, 0, 6)
s9 = (4, 1, 3, 5, 7, 2, 0, 8, 6)
s10 = (4, 1, 3, 0, 7, 2, 5, 8, 6)
s11 = (4, 1, 3, 7, 0, 2, 5, 8, 6)
s12 = (4, 1, 3, 7, 2, 0, 5, 8, 6)
s13 = (4, 1, 3, 7, 2, 6, 5, 8, 0)
s14 = (4, 1, 3, 7, 2, 6, 5, 0, 8)
s15 = (4, 1, 3, 7, 2, 6, 0, 5, 8)
s16 = (4, 1, 3, 0, 2, 6, 7, 5, 8)
s17 = (0, 1, 3, 4, 2, 6, 7, 5, 8)
s18 = (1, 0, 3, 4, 2, 6, 7, 5, 8)
s19 = (1, 2, 3, 4, 0, 6, 7, 5, 8)
s20 = (1, 2, 3, 4, 5, 6, 7, 0, 8)
s21 = (1, 2, 3, 4, 5, 6, 7, 8, 0)

color_dict = {
    1: '#404040',
    2: '#595959',
    3: '#737373',
    4: '#8c8c8c',
    5: '#a6a6a6',
    6: '#bfbfbf',
    7: '#d9d9d9',
    8: '#f2f2f2',
    0: ''
}

for i in range(1, 22):
    s = f"s{i}"
    table = widgets.HTML(
        value=f'''
        <table style="border-collapse: collapse; border: 2px solid black; margin: auto;">
            <tr>
                <td style="width: 50px; height: 50px; text-align: center; background-color: {color_dict[eval(s)[0]]};">{eval(s)[0]}</td>
                <td style="width: 50px; height: 50px; text-align: center; background-color: {color_dict[eval(s)[1]]};">{eval(s)[1]}</td>
                <td style="width: 50px; height: 50px; text-align: center; background-color: {color_dict[eval(s)[2]]};">{eval(s)[2]}</td>
            </tr>
            <tr>
                <td style="width: 50px; height: 50px; text-align: center; background-color: {color_dict[eval(s)[3]]};">{eval(s)[3]}</td>
                <td style="width: 50px; height: 50px; text-align: center; background-color: {color_dict[eval(s)[4]]};">{eval(s)[4]}</td>
                <td style="width: 50px; height: 50px; text-align: center; background-color: {color_dict[eval(s)[5]]};">{eval(s)[5]}</td>
            </tr>
            <tr>
                <td style="width: 50px; height: 50px; text-align: center; background-color: {color_dict[eval(s)[6]]};">{eval(s)[6]}</td>
                <td style="width: 50px; height: 50px; text-align: center; background-color: {color_dict[eval(s)[7]]};">{eval(s)[7]}</td>
                <td style="width: 50px; height: 50px; text-align: center; background-color: {color_dict[eval(s)[8]]};">{eval(s)[8]}</td>
            </tr>
        </table>
        ''',
    )
    display(table)
    print('')
    print('')


HTML(value='\n        <table style="border-collapse: collapse; border: 2px solid black; margin: auto;">\n     …

HTML(value='\n        <table style="border-collapse: collapse; border: 2px solid black; margin: auto;">\n     …

HTML(value='\n        <table style="border-collapse: collapse; border: 2px solid black; margin: auto;">\n     …

HTML(value='\n        <table style="border-collapse: collapse; border: 2px solid black; margin: auto;">\n     …

HTML(value='\n        <table style="border-collapse: collapse; border: 2px solid black; margin: auto;">\n     …

...

HTML(value='\n        <table style="border-collapse: collapse; border: 2px solid black; margin: auto;">\n     …

HTML(value='\n        <table style="border-collapse: collapse; border: 2px solid black; margin: auto;">\n     …

HTML(value='\n        <table style="border-collapse: collapse; border: 2px solid black; margin: auto;">\n     …

HTML(value='\n        <table style="border-collapse: collapse; border: 2px solid black; margin: auto;">\n     …

HTML(value='\n        <table style="border-collapse: collapse; border: 2px solid black; margin: auto;">\n     …